In [2]:
import os
import re
import string
from nltk.corpus import stopwords
from snowballstemmer import stemmer
import nltk
import pandas as pd


In [3]:
def normalize_text(s):
    """Lower text and remove punctuation, Singlecaracter,number and extra whitespace."""
    def remove_number(text):
            return  re.sub(r'\d+','',text)
    def white_space_fix(text):
            return ' '.join(text.split())
    def remove_punc(text):
            exclude = set(string.punctuation)
            return ''.join(ch for ch in text if ch not in exclude)
    def remove_Singlecaracter(text):
            return ' '.join( [w for w in text.split() if len(w)>1] )
    def lower(text):
            return text.lower()
    print()
    return remove_Singlecaracter(white_space_fix(remove_number(remove_punc(lower(s))))) 

In [4]:
def Normalization_File(String):
    stops = set(stopwords.words('arabic'))
    ar_stemmer = stemmer("arabic")
    text=nltk.word_tokenize(normalize_text(String))  
    fileText=[ar_stemmer.stemWord(w) for w in text if not w in stops]
    return " ".join(fileText)

In [5]:
QueryList={}
# QueryList['eng']=[
#     {"Q1":'Smart','Grid','connection'},
#     {"Q2":'Financial','corruption'},
#     {"Q3":'Phone','service'},
# ]
QueryList['ar']=[
    {"Q1":['المعلوماتي','الأمن']},
    {"Q2":['المعارضة','الفرضيات']},
    {"Q3":['الطرق','حوادث']},
]


In [6]:
def manSearch_arab(path,doc_pertinents=None):
    encodings = ['utf-8', 'windows-1256','ISO-8859-1']  
    for file in os.listdir(path):
                if file.endswith(".txt"):
                    file_path = f"{path}/{file}"
                    for e in encodings:
                        try:
                            with open(file_path, 'r', encoding=e) as f:
                                text=Normalization_File(f.read())
                        except UnicodeDecodeError:
                            continue
                        else:
                            break
                    doc_pertinents['title'].append(file)
                    for Q_obj in QueryList['ar']:
                        for key, value in Q_obj.items():
                            if(len([1 for word in value if Normalization_File(word) in text])>0):
                                doc_pertinents[key].append(1)
                            else:
                                doc_pertinents[key].append(0)
    return doc_pertinents

In [7]:
doc_pertinents={"title":[],"Q1":[],"Q2":[],"Q3":[]}
doc_pertinents=manSearch_arab("/home/lounis/Documents/M2/RI/TP2/Corpus/Arabic",doc_pertinents)


In [8]:
cpt=0
for x in doc_pertinents["Q3"]:
    if(x>=1): 
        cpt=cpt+1
print(cpt)
    



133


In [9]:
df=pd.DataFrame(doc_pertinents,index=doc_pertinents["title"],columns=["Q1","Q2","Q3"]).sort_index()
df.to_excel('./Result/RechercheManuelle.xlsx')
# Excel=df.to_excel("./")